In [2]:
import mysql.connector
import pandas as pd

# Connect to your database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',  # change this
    database='apartment_rent'
)

cursor = conn.cursor()


In [27]:
train_df = pd.read_csv('train_address_derived_filled_before_SQL.csv', encoding='utf-8')

In [5]:
train_df.to_csv('train_address_derived_filled_before_SQL1.csv', index=False, encoding='utf-8-sig')

In [53]:
train_df= pd.read_csv('train_address_derived_filled_before_SQL1.csv', encoding='utf-8')

In [54]:
train_df.columns = train_df.columns.str.lower().str.strip()  #

In [55]:
train_df.columns

Index(['new_construction', 'elevator', 'floors_in_the_building', 'floor_area',
       'number_of_rooms', 'number_of_bathrooms', 'ceiling_height', 'floor',
       'children_are_welcome', 'pets_allowed', 'utility_payments', 'address',
       'monthly_price_usd', 'log_price', 'duration_encoded',
       'construction_type_encoded', 'balcony_encoded', 'furniture_encoded',
       'renovation_encoded', 'address_translated', 'armenian_address',
       'russian_address', 'lat', 'lon', 'city', 'district'],
      dtype='object')

In [56]:
# train_df = train_df.drop(['log_price'], axis=1)

In [57]:
def reverse_encoding(df):
    # Reverse mapping for duration
    duration_reverse_map = {0: 'daily', 1: 'monthly'}
    df['duration'] = df['duration_encoded'].map(duration_reverse_map)

    # Reverse mapping for other categorical features
    categorical_mappings = {
        'construction_type': {0: 'stone', 1: 'cassette', 2: 'panels', 3: 'monolith', 4: 'bricks'},
        'balcony': {0: 'not_available', 1: 'closed_balcony', 2: 'open_balcony', 3: 'multiple_balconies'},
        'furniture': {0: 'not_available', 1: 'partial_furniture', 2: 'by_agreement', 3: 'available'},
        'renovation': {
            0: 'no_renovation', 1: 'partial_renovation', 2: 'old_renovation',
            3: 'cosmetic_renovation', 4: 'euro_renovation', 5: 'major_renovation',
            6: 'designer_renovation'
        }
    }

    for col, reverse_map in categorical_mappings.items():
        encoded_col = f"{col}_encoded"
        if encoded_col in df.columns:
            df[col] = df[encoded_col].map(reverse_map)

     # Drop encoded and derived columns
    cols_to_drop = [
        'monthly_price_usd',
        'duration_encoded',
        'construction_type_encoded',
        'balcony_encoded',
        'furniture_encoded',
        'renovation_encoded'
    ]
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

    return df

In [58]:
train_df_1 = reverse_encoding(train_df)

In [31]:
train_df_1

,new_construction,elevator,floors_in_the_building,floor_area,number_of_rooms,number_of_bathrooms,ceiling_height,floor,children_are_welcome,pets_allowed,...,russian_address,lat,lon,city,district,duration,construction_type,balcony,furniture,renovation
0,0,0,5,41,2,1,2.70,5,11,11,...,0,39.956663,44.547101,artashat,non_yerevan,daily,stone,open_balcony,available,major_renovation
1,0,0,5,76,4,1,2.70,4,11,11,...,0,40.161840,44.291115,armavir,non_yerevan,monthly,stone,open_balcony,available,euro_renovation
2,0,0,5,78,3,1,2.70,3,11,11,...,0,40.161840,44.291115,armavir,non_yerevan,monthly,stone,closed_balcony,available,major_renovation
3,1,1,12,40,1,1,2.50,9,0,0,...,0,40.531154,44.720436,tsaghkadzor,non_yerevan,daily,stone,open_balcony,available,euro_renovation
4,0,0,4,100,3,1,2.60,3,11,11,...,0,40.828308,43.823557,gyumri,non_yerevan,monthly,bricks,open_balcony,available,cosmetic_renovation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31236,0,1,11,52,1,1,3.00,9,11,10,...,0,40.181857,44.511272,yerevan,kentron,monthly,panels,not_available,available,major_renovation
31237,1,1,12,200,3,2,3.00,10,11,12,...,0,40.185157,44.520600,yerevan,kentron,monthly,monolith,multiple_balconies,available,euro_renovation
31238,0,1,5,70,2,1,3.00,2,11,10,...,0,40.182062,44.520884,yerevan,kentron,monthly,stone,not_available,available,euro_renovation
31239,0,0,5,42,1,1,2.75,4,11,10,...,0,40.182062,44.520884,yerevan,kentron,monthly,stone,open_balcony,available,designer_renovation


In [48]:
original_df = pd.read_csv('train_address_derived1_filled2..csv', encoding='utf-8')

In [59]:
original_df = pd.read_csv('datasets/summative/apartment_for_rent_train.csv', encoding='utf-8')

In [32]:
train_df_1.to_csv('check.csv', index=False, encoding='utf-8-sig')

In [44]:
import pandas as pd
import re

# Function to clean and standardize delimiters and structure
def standardize_address_format(address):
    if pd.isnull(address):
        return address
    
    # Replace › with ,
    address = address.replace("›", ",")
    
    # Remove redundant whitespaces and standardize commas
    address = re.sub(r'\s*,\s*', ', ', address.strip())  # Normalize commas with single space
    address = re.sub(r'\s+', ' ', address)               # Remove extra spaces
    return address

In [45]:
import pandas as pd

def map_original_info(cleaned_df, original_df):
    # Columns to compare for matching
    columns_to_check = [
        'new_construction', 'elevator', 'floors_in_the_building', 'floor_area',
        'number_of_rooms', 'number_of_bathrooms', 'ceiling_height', 'floor',
        'children_are_welcome', 'pets_allowed', 'utility_payments',
        'address', 'log_price', 'balcony', 'furniture', 'renovation'
    ]
    original_df["address"] = original_df["address"].apply(standardize_address_format)

    float_cols = ['log_price', 'ceiling_height', 'floor_area']
    for col in float_cols:
        cleaned_df[col] = cleaned_df[col].round(3)
        original_df[col] = original_df[col].round(3)

    str_cols = ['address', 'balcony', 'furniture', 'renovation']

    for col in str_cols:
        cleaned_df[col] = cleaned_df[col].str.strip().str.lower()
        original_df[col] = original_df[col].str.strip().str.lower()

    # Columns to bring from the original dataset
    original_fields = [
        'currency', 'reg_id', 'datetime', 'price',
        'amenities', 'appliances', 'parking'
    ]

    # Ensure all required columns exist in both dataframes
    for col in columns_to_check:
        if col not in cleaned_df.columns or col not in original_df.columns:
            raise ValueError(f"Missing column '{col}' in one of the DataFrames.")

    # Perform the merge on the specified columns
    merged_df = cleaned_df.merge(
        original_df[original_fields + columns_to_check],
        on=columns_to_check,
        how='left',
        suffixes=('', '_original')
    )

    # Add a column to flag matched rows
    merged_df['original_matched'] = ~merged_df['reg_id'].isna()

    return merged_df


In [60]:
train_df_2 = map_original_info(train_df_1, original_df)

KeyError: 'address'

In [47]:
train_df_2.to_csv('train_address_derived_filled_before_SQL2.csv', index=False, encoding='utf-8-sig')

In [61]:
train_df_3 = pd.read_csv('train_address_derived_filled_before_SQL2.csv', encoding='utf-8')

In [62]:
train_df_3

,new_construction,elevator,floors_in_the_building,floor_area,number_of_rooms,number_of_bathrooms,ceiling_height,floor,children_are_welcome,pets_allowed,...,furniture,renovation,currency,reg_id,datetime,price,amenities,appliances,parking,original_matched
0,0,0,5,41,2,1,2.70,5,11,11,...,available,major_renovation,AMD,3.0,24/11/2022,20000.0,none,none,none,True
1,0,0,5,76,4,1,2.70,4,11,11,...,available,euro_renovation,AMD,2.0,24/11/2022,270000.0,none,none,none,True
2,0,0,5,78,3,1,2.70,3,11,11,...,available,major_renovation,AMD,2.0,15/12/2022,160000.0,none,none,none,True
3,1,1,12,40,1,1,2.50,9,0,0,...,available,euro_renovation,AMD,4.0,24/11/2022,20000.0,none,none,none,True
4,0,0,4,100,3,1,2.60,3,11,11,...,available,cosmetic_renovation,AMD,5.0,15/12/2022,200000.0,none,none,none,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31698,0,1,11,52,1,1,3.00,9,11,10,...,available,major_renovation,AMD,1.0,15/12/2022,370000.0,none,none,none,True
31699,1,1,12,200,3,2,3.00,10,11,12,...,available,euro_renovation,USD,1.0,15/12/2022,2000.0,none,none,none,True
31700,0,1,5,70,2,1,3.00,2,11,10,...,available,euro_renovation,AMD,1.0,15/12/2022,490000.0,none,none,none,True
31701,0,0,5,42,1,1,2.75,4,11,10,...,available,designer_renovation,AMD,1.0,15/12/2022,400000.0,none,none,none,True


In [64]:
train_df_3 = train_df_3[train_df_3['original_matched'] == True]

In [65]:
train_df_3.drop(['original_matched'], axis=1, inplace=True)

C:\Users\kelst\AppData\Local\Temp\ipykernel_38284\744412787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_3.drop(['original_matched'], axis=1, inplace=True)


In [66]:
train_df_3.to_csv('train_address_derived_filled_before_SQL3.csv', index=False, encoding='utf-8-sig')

In [67]:
train_df_4 = pd.read_csv('train_address_derived_filled_before_SQL3.csv', encoding='utf-8')

In [92]:
import re
import pandas as pd

# Match numbers like 13, 13a, 37/1 — skip ordinals like 1st, 2nd, etc.
number_pattern = re.compile(r'^(?!\d*(?:st|nd|rd|th))[\d]+[a-zA-Z/]*$')

def parse_block_info(address):
    block = None
    block_number = None

    if pd.isna(address):
        return block, block_number

    parts = address.split(',')

    for part in parts:
        if 'block' in part:
            block_part = part.strip().lower().replace(' ', '_')
            block_parts = block_part.split('_')
            for i, token in enumerate(block_parts):
                if 'block' in token:
                    # Try joining next tokens for patterns like 11/1 or 14a
                    for j in range(i + 1, len(block_parts)):
                        candidate = ''.join(block_parts[i + 1:j + 1])
                        if number_pattern.match(candidate):
                            block_number = candidate
                            block = '_'.join(block_parts[:j + 1])
                            return block, block_number
                    block = '_'.join(block_parts)
                    return block, block_number
    return block, block_number

def apply_street_block_parsing(df):
    df['street'] = None
    df['street_number'] = None
    df['block'] = None
    df['block_number'] = None

    for index, address in df['address_translated'].items():
        street = street_number = block = block_number = None

        if pd.isna(address):
            df.at[index, 'street'] = street
            df.at[index, 'street_number'] = street_number
            df.at[index, 'block'] = block
            df.at[index, 'block_number'] = block_number
            continue

        parts = address.lower().split('_')

        # Fix combinations like '29_a' → '29a'
        merged_parts = []
        i = 0
        while i < len(parts):
            if i + 1 < len(parts) and parts[i].isdigit() and len(parts[i + 1]) == 1 and parts[i + 1].isalpha():
                merged_parts.append(parts[i] + parts[i + 1])
                i += 2
            else:
                merged_parts.append(parts[i])
                i += 1

        # -------- Street / Avenue Parsing --------
        street_terms = ['street', 'avenue']
        street_idx = [i for i, part in enumerate(merged_parts) if part in street_terms]

        if merged_parts and number_pattern.match(merged_parts[0]):
            street_number = merged_parts[0]
            if street_idx:
                idx = street_idx[0]
                street = '_'.join(merged_parts[1:idx + 1])  # exclude number
        elif street_idx:
            idx = street_idx[0]
            if idx + 1 < len(merged_parts) and number_pattern.match(merged_parts[idx + 1]):
                street_number = merged_parts[idx + 1]
                street = '_'.join(merged_parts[:idx + 1])
            else:
                street = '_'.join(merged_parts[:idx + 1])

        # -------- Block Parsing --------
        block, block_number = parse_block_info(address)

        # Assign parsed values
        df.at[index, 'street'] = street
        df.at[index, 'street_number'] = street_number
        df.at[index, 'block'] = block
        df.at[index, 'block_number'] = block_number

    return df


In [93]:
# Re-load the file since environment was reset
file_path = "train_address_derived_filled_before_SQL3.csv"
df = pd.read_csv(file_path)

# Apply parsing function
df = apply_street_block_parsing(df)

In [94]:
df

,new_construction,elevator,floors_in_the_building,floor_area,number_of_rooms,number_of_bathrooms,ceiling_height,floor,children_are_welcome,pets_allowed,...,reg_id,datetime,price,amenities,appliances,parking,street,street_number,block,block_number
0,0,0,5,41,2,1,2.70,5,11,11,...,3.0,24/11/2022,20000.0,none,none,none,None,None,None,None
1,0,0,5,76,4,1,2.70,4,11,11,...,2.0,24/11/2022,270000.0,none,none,none,None,None,None,None
2,0,0,5,78,3,1,2.70,3,11,11,...,2.0,15/12/2022,160000.0,none,none,none,None,None,None,None
3,1,1,12,40,1,1,2.50,9,0,0,...,4.0,24/11/2022,20000.0,none,none,none,None,None,None,None
4,0,0,4,100,3,1,2.60,3,11,11,...,5.0,15/12/2022,200000.0,none,none,none,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31152,0,1,11,52,1,1,3.00,9,11,10,...,1.0,15/12/2022,370000.0,none,none,none,None,None,None,None
31153,1,1,12,200,3,2,3.00,10,11,12,...,1.0,15/12/2022,2000.0,none,none,none,None,None,None,None
31154,0,1,5,70,2,1,3.00,2,11,10,...,1.0,15/12/2022,490000.0,none,none,none,None,None,None,None
31155,0,0,5,42,1,1,2.75,4,11,10,...,1.0,15/12/2022,400000.0,none,none,none,None,None,None,None


In [95]:
df.to_csv('check.csv', index=False, encoding='utf-8-sig')

In [97]:
train_db = pd.read_csv('train_db.csv', encoding='utf-8')

C:\Users\kelst\AppData\Local\Temp\ipykernel_38284\985007019.py:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  train_db = pd.read_csv('train_db.csv', encoding='utf-8')


In [98]:
train_db

,new_construction,elevator,floors_in_the_building,floor_area,number_of_rooms,number_of_bathrooms,ceiling_height,floor,children_are_welcome,pets_allowed,...,reg_id,datetime,price,amenities,appliances,parking,street,street_number,block,block_number
0,0,0,5,41,2,1,2.70,5,11,11,...,3.0,24/11/2022,20000.0,none,none,none,NaN,NaN,NaN,NaN
1,0,0,5,76,4,1,2.70,4,11,11,...,2.0,24/11/2022,270000.0,none,none,none,NaN,NaN,NaN,NaN
2,0,0,5,78,3,1,2.70,3,11,11,...,2.0,15/12/2022,160000.0,none,none,none,NaN,NaN,NaN,NaN
3,1,1,12,40,1,1,2.50,9,0,0,...,4.0,24/11/2022,20000.0,none,none,none,NaN,NaN,NaN,NaN
4,0,0,4,100,3,1,2.60,3,11,11,...,5.0,15/12/2022,200000.0,none,none,none,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31152,0,1,11,52,1,1,3.00,9,11,10,...,1.0,15/12/2022,370000.0,none,none,none,NaN,NaN,NaN,NaN
31153,1,1,12,200,3,2,3.00,10,11,12,...,1.0,15/12/2022,2000.0,none,none,none,NaN,NaN,NaN,NaN
31154,0,1,5,70,2,1,3.00,2,11,10,...,1.0,15/12/2022,490000.0,none,none,none,NaN,NaN,NaN,NaN
31155,0,0,5,42,1,1,2.75,4,11,10,...,1.0,15/12/2022,400000.0,none,none,none,NaN,NaN,NaN,NaN


In [102]:
train_db1 = train_db.drop(['armenian_address', 'russian_address', 'log_price'], axis=1)

In [103]:
train_db1.to_csv('train_db1.csv', index=False, encoding='utf-8-sig')

In [104]:
train_db2 = pd.read_csv('train_db1.csv', encoding='utf-8')

C:\Users\kelst\AppData\Local\Temp\ipykernel_38284\1651347067.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  train_db2 = pd.read_csv('train_db1.csv', encoding='utf-8')


In [105]:
import pandas as pd
import numpy as np

def prepare_for_sql_insertion(df):
    # 1. Handle missing values
    df = df.replace({np.nan: None, 'nan': None, '': None})
    
    # 2. Ensure proper dtypes for numeric fields (cast or clean)
    numeric_cols = [
        'new_construction', 'elevator', 'floors_in_the_building', 'floor_area',
        'number_of_rooms', 'number_of_bathrooms', 'ceiling_height', 'floor',
        'children_are_welcome', 'pets_allowed', 'utility_payments',
        'lat', 'lon', 'reg_id', 'price'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 3. Convert datetime fields
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
        df['datetime'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # 4. Clean and standardize string fields
    string_cols = [
        'address', 'address_translated', 'city', 'district', 'duration',
        'construction_type', 'balcony', 'furniture', 'renovation', 'currency',
        'amenities', 'appliances', 'parking',
        'street', 'street_number', 'block', 'block_number'
    ]
    for col in string_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().replace('nan', None)
    
    # 5. Clean multivalued strings: remove leading/trailing _, split by comma, and standardize
    multivalue_cols = ['amenities', 'appliances', 'parking']
    for col in multivalue_cols:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: ','.join(
                [item.strip('_ ').lower() for item in str(x).split(',')] if x else []
            ) if x else None)

    return df


In [106]:
train_db3 = prepare_for_sql_insertion(train_db2 )

C:\Users\kelst\AppData\Local\Temp\ipykernel_38284\2141864933.py:21: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')


In [108]:
train_db3.to_csv('check_mate.csv', index=False, encoding='utf-8-sig')

In [113]:
train_db4 = pd.read_csv('check_mate.csv', encoding='utf-8')

C:\Users\kelst\AppData\Local\Temp\ipykernel_38284\1036459858.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  train_db4 = pd.read_csv('check_mate.csv', encoding='utf-8')


In [114]:
train_db4 ['apartment_no'] = range(1, len(df) + 1)

In [129]:
import pandas as pd
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',  # Replace with your password
    database='apartment_rent'
)
cursor = conn.cursor()

# Load your full cleaned dataset
df = pd.read_csv("train_db3.csv")
df['apartment_no'] = range(1, len(df) + 1)

def get_or_create_address(cursor, row):
    cursor.execute("""
        SELECT Address_No FROM Address
        WHERE Raw_Address = %s AND Translated_Address = %s AND City = %s AND District = %s
          AND Lat = %s AND Lon = %s
    """, (
        row['address'], row['address_translated'], row['city'], row['district'],
        row['lat'], row['lon']
    ))
    
    result = cursor.fetchone()
    if result:
        return result[0]

    cursor.execute("""
        INSERT INTO Address (
            Raw_Address, Translated_Address, Street, Street_Number,
            Block, Block_Number, District, City, Lat, Lon
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['address'], row['address_translated'], row['street'], row['street_number'],
        row['block'], row['block_number'], row['district'], row['city'], row['lat'], row['lon']
    ))
    conn.commit()
    return cursor.lastrowid


# === Step 1: Insert all basic apartment fields with dynamic address_no ===
for idx, row in df.iterrows():
    row = row.where(pd.notnull(row), None)
    address_no = get_or_create_address(cursor, row)
     # Replace NaN with None
    
    cursor.execute("""
        INSERT INTO Apartment (
            Apartment_No, Address_No, Rent_Amount, Currency, Duration,
            Renovation, New_Construction, Construction_Type, Furniture,
            Floor, Floor_Area, Elevator, Floor_In_Building, Number_Of_Rooms,
            Number_Of_Bathrooms, Ceiling_Height, Balcony, Children_Welcome,
            Pets_Allowed, Utility_Payments, Datetime
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['apartment_no'], address_no, row['price'], row['currency'], row['duration'],
        row['renovation'], row['new_construction'], row['construction_type'], row['furniture'],
        row['floor'], row['floor_area'], row['elevator'], row['floors_in_the_building'],
        row['number_of_rooms'], row['number_of_bathrooms'], row['ceiling_height'], row['balcony'],
        row['children_are_welcome'], row['pets_allowed'], row['utility_payments'], row['datetime']
    ))

conn.commit()

# === Step 2: Helper to insert category records ===
def get_or_create(cursor, table, description):
    cursor.execute(f"SELECT {table.upper()}_NO FROM {table.upper()} WHERE DESCRIPTION = %s", (description,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute(f"INSERT INTO {table.upper()} (DESCRIPTION) VALUES (%s)", (description,))
    conn.commit()
    return cursor.lastrowid

def insert_link(apartment_no, item_no, link_table, item_field):
    cursor.execute(
        f"INSERT INTO {link_table} (APARTMENT_NO, {item_field}) VALUES (%s, %s)",
        (apartment_no, item_no)
    )

# === Step 3: Insert multi-label values ===
def process_multilabel(df, column_name, table, link_table, item_field):
    for idx, row in df.iterrows():
        apartment_no = row['apartment_no']
        items = str(row[column_name]).split(',')
        for item in items:
            clean_item = item.strip().lstrip('_')
            if clean_item:
                item_no = get_or_create(cursor, table, clean_item)
                insert_link(apartment_no, item_no, link_table, item_field)
    conn.commit()

# Process multi-label fields
process_multilabel(df, 'appliances', 'appliance', 'apartment_appliance', 'appliance_no')
process_multilabel(df, 'amenities', 'amenity', 'apartment_amenity', 'amenity_no')
process_multilabel(df, 'parking', 'parking', 'apartment_parking', 'parking_no')


C:\Users\kelst\AppData\Local\Temp\ipykernel_38284\24709019.py:14: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_db3.csv")


In [116]:
train_db4

,new_construction,elevator,floors_in_the_building,floor_area,number_of_rooms,number_of_bathrooms,ceiling_height,floor,children_are_welcome,pets_allowed,...,datetime,price,amenities,appliances,parking,street,street_number,block,block_number,apartment_no
0,0,0,5,41,2,1,2.70,5,11,11,...,2022-11-24 00:00:00,20000.0,none,none,none,NaN,NaN,NaN,NaN,1
1,0,0,5,76,4,1,2.70,4,11,11,...,2022-11-24 00:00:00,270000.0,none,none,none,NaN,NaN,NaN,NaN,2
2,0,0,5,78,3,1,2.70,3,11,11,...,2022-12-15 00:00:00,160000.0,none,none,none,NaN,NaN,NaN,NaN,3
3,1,1,12,40,1,1,2.50,9,0,0,...,2022-11-24 00:00:00,20000.0,none,none,none,NaN,NaN,NaN,NaN,4
4,0,0,4,100,3,1,2.60,3,11,11,...,2022-12-15 00:00:00,200000.0,none,none,none,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31152,0,1,11,52,1,1,3.00,9,11,10,...,2022-12-15 00:00:00,370000.0,none,none,none,NaN,NaN,NaN,NaN,31153
31153,1,1,12,200,3,2,3.00,10,11,12,...,2022-12-15 00:00:00,2000.0,none,none,none,NaN,NaN,NaN,NaN,31154
31154,0,1,5,70,2,1,3.00,2,11,10,...,2022-12-15 00:00:00,490000.0,none,none,none,NaN,NaN,NaN,NaN,31155
31155,0,0,5,42,1,1,2.75,4,11,10,...,2022-12-15 00:00:00,400000.0,none,none,none,NaN,NaN,NaN,NaN,31156


In [127]:
import mysql.connector
import pandas as pd

# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',  # Replace with your actual password
    database='apartment_rent'
)

# Define query
query = """
SELECT *
FROM Apartment
WHERE Rent_Amount <= 1000
  AND Elevator = '1'
  AND Currency = 'USD';
"""

# Load into DataFrame
df = pd.read_sql(query, conn)

# Export to CSV
df.to_csv('filtered_apartments_task2bii.csv', index=False)

# Close connection
conn.close()

print("Exported to filtered_apartments.csv")


Exported to filtered_apartments.csv


C:\Users\kelst\AppData\Local\Temp\ipykernel_38284\3977123388.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [128]:
import mysql.connector
import pandas as pd

# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',  # Replace with your actual password
    database='apartment_rent'
)

# Define query
query = """
SELECT * FROM Apartment_Appliance;;
"""

# Load into DataFrame
df = pd.read_sql(query, conn)

# Export to CSV
df.to_csv('checking.csv', index=False)

# Close connection
conn.close()

print("Exported to filtered_apartments.csv")

C:\Users\kelst\AppData\Local\Temp\ipykernel_38284\168129126.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Exported to filtered_apartments.csv
